# Анализ базы данных сервиса для чтения книг

**Цель**

Сформулировать ценностное предложение для нового продукта, на основе анализа информации о книгах, издательствах, авторах и пользовательских обзорах книг.

**Задачи**

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (за исключением брошюр);
- Определить автора с самой высокой средней оценкой книг (учесть только книги с 50 и более оценками);
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем первые строки всех таблиц

In [3]:
query = '''
    SELECT *
    FROM books
    LIMIT 5;
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
    SELECT *
    FROM authors
    LIMIT 5;
'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''
    SELECT *
    FROM ratings
    LIMIT 5;
'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''
    SELECT *
    FROM reviews
    LIMIT 5;
'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
query = '''
    SELECT *
    FROM publishers
    LIMIT 5;
'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


##  Количество книг, вышедших после 1 января 2000 года

In [8]:
query = '''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01';
'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

## Количество обзоров и средняя оценка каждой книги

In [18]:
query = '''
    SELECT b.book_id,
           b.title,
           COUNT(DISTINCT re.review_id),
           AVG(ra.rating)
    FROM reviews AS re
    FULL JOIN ratings AS ra ON re.book_id=ra.book_id
    RIGHT JOIN books AS b ON re.book_id=b.book_id
    GROUP BY b.book_id;
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,count,avg
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


Всего в базе 1000 книг.

##  Издательство с наибольшим числом книг толще 50 страниц

In [ ]:
query = '''
    SELECT p.publisher,
           COUNT(b.book_id)
    FROM books AS b
    JOIN publishers AS p ON b.publisher_id=p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher_id
    ORDER BY COUNT(b.book_id) DESC
    LIMIT 1;
'''

pd.io.sql.read_sql(query, con = engine)

Издательство с наибольшим числом книг толще 50 страниц - Penguin Books c 42 книгами.

##  Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)

In [ ]:
query = '''
    SELECT a.author,
           AVG(r.rating)
    FROM authors AS a
    JOIN books AS bo ON a.author_id=bo.author_id
    JOIN ratings AS r ON bo.book_id=r.book_id
    WHERE bo.book_id IN (SELECT b.book_id
                        FROM books AS b
                        JOIN ratings AS r ON b.book_id=r.book_id
                        GROUP BY b.book_id
                        HAVING COUNT(r.rating_id) >= 50)
    GROUP BY a.author_id
    ORDER BY AVG(r.rating) DESC
    LIMIT 1;
'''

pd.io.sql.read_sql(query, con = engine)

Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [ ]:
query = '''
WITH
r AS (SELECT COUNT(review_id) AS reviews_count
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 50)
      GROUP BY username)
SELECT AVG(reviews_count)
FROM r;
'''

pd.io.sql.read_sql(query, con = engine)

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.3